In [1]:
%matplotlib inline

In [1]:
import os
import re
import xml.etree.ElementTree as ET
import json
from pathlib import Path
from bs4 import BeautifulSoup

In [3]:
from pymongo import MongoClient
client = MongoClient()
db = client.stackoverflow
posts = db.posts

In [9]:
def sanitise_xml(s):
    """
    Use BeautifulSoup to extract all text within HTML/XML tags, except those in <code> blocks.
    Remove the line breaks, replace unicode character for space
    
    Parameters
    ----------
    s - UTF-8 string
    
    Returns
    -------
    Cleaned up string    
    """
    soup = BeautifulSoup(s, 'lxml')
    for tag in soup.find_all('code'):
        tag.replace_with('')
        
    return soup.get_text().replace('\n', '').replace('\xa0', ' ')

In [10]:
def preprocess_text(data):
    """
    Clean the text body of the question/answers by removing tags and code,
    and split the Tags into a list.
    
    Parameter
    ---------
    data - dictionary, data extracted from an XML line using ElementTree
    
    Returns
    -------
    Processed dictionary
    """
    if 'Body' in data:
        data['Body'] = sanitise_xml(data['Body'])
    
    if 'Tags' in data:
        data['Tags'] = re.findall('\<([^>]+)', data['Tags'])
        
    return data

In [11]:
PATH = Path('/Users/sueliu/Downloads/stackoverflow/posts')

In [12]:
FIRST_FILE = Path(PATH/"postsaa")

In [13]:
new_posts = []
counter = 0
with open(FIRST_FILE) as f:
    for line in f:
        data = preprocess_text(ET.fromstring(line.strip()).attrib)
        new_posts.append(data)
        counter += 1

In [14]:
result = posts.insert_many(new_posts)